In [111]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

### 0) Configuração de pré-pesquisa
Aqui será configurado as informaçoes fornecidas para a pesquisa, como a data de inicio e fim da busca, tamanho da visualização e UF(Unidade Federativa) que será pesquisado os discursos.

In [112]:
#formato de data = {dd, mm, aaaa}
data_inicio = [28, 3, 2022]
data_fim = [28, 4, 2022]
#tamanho = quantidade de linhas de informações por página (limite 1788 pq? não sabemos)
tamanho = 100
current_page = 1
#Estado onde será realizada a pesquisa ('' para todos)
UF = ''

url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
      

In [113]:
html = requests.get(url)

content = html.content

soup = BeautifulSoup(content, 'html.parser')

In [114]:
quantidade_de_discursos= []
qpt = 0 #quantidade de paginas totais

for links in soup.find_all('span', attrs={'class':"visualStrong"}):
    quantidade_de_discursos.append(links.contents)
qpt= int(((int((quantidade_de_discursos[len(quantidade_de_discursos)-1][0]).replace('.','')))/tamanho)+1)
qpt

20

In [136]:

def UrlToSoup(url):
    html = requests.get(url)
    content = html.content
    return BeautifulSoup(content, 'html.parser')

### 1) Criar um data frame com as informações do site

In [116]:
url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF={UF}&dtInicio={data_inicio[0]}%2F{data_inicio[1]}%2F{data_inicio[2]}&dtFim={data_fim[0]}%2F{data_fim[1]}%2F{data_fim[2]}&txSessao=&listaTipoFala=&listaFaseSessao=&listaTipoInterv=&txAparteante=&txTexto=&txSumario=&txIndexacao=&BasePesq=plenario&CampoOrdenacao=dtSessao&PageSize={tamanho}&TipoOrdenacao=ASC&btnPesqAvan=Pesquisar'

html = requests.get(url)

content = html.content

soup = BeautifulSoup(content, 'html.parser')

print(url)

https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?txOrador=&txPartido=&txUF=&dtInicio=28%2F3%2F2022&dtFim=28%2F4%2F2022&txSessao=&listaTipoFala=&listaFaseSessao=&listaTipoInterv=&txAparteante=&txTexto=&txSumario=&txIndexacao=&BasePesq=plenario&CampoOrdenacao=dtSessao&PageSize=100&TipoOrdenacao=ASC&btnPesqAvan=Pesquisar


In [128]:
data_frame = [None]*qpt
for i in range(qpt):
    current_page = i+1
    url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
    data_frame[i] = pd.read_html(url)


In [182]:
data_frame[0][0]

,Data,Sessão,Fase,Discurso,Sumário,Orador,Hora,Publicação
0,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022
1,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022
2,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022
3,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022
4,28/04/2022,013.4.56.N,ORDEM DO DIA,NaN,NaN,"AFONSO FLORENCE, PT-BA",12h37,DCN 29/04/2022
...,...,...,...,...,...,...,...,...
95,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"REINHOLD STEPHANES JUNIOR, PSD-PR",15h52,DCD 30/03/2022
96,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"FRANCO CARTAFINA, PP-MG",15h56,DCD 30/03/2022
97,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"MARCEL VAN HATTEM, NOVO-RS",16h00,DCD 30/03/2022
98,29/03/2022,25.2022,BREVES COMUNICAÇÕES,NaN,NaN,"DANIEL SILVEIRA, UNIÃO-RJ",16h04,DCD 30/03/2022


### 2) Criar uma lista com todos os links para os discursos

In [171]:
link_infos= []
soup = []
temp = []
for i in range(qpt):
        current_page = i+1
        url = f'https://www.camara.leg.br/internet/SitaqWeb/ResultadoPesquisaDiscursos.asp?CurrentPage={current_page}&txIndexacao=&BasePesq=plenario&txOrador=&txPartido=&dtInicio={data_inicio[0]}/{data_inicio[1]}/{data_inicio[2]}&dtFim={data_fim[0]}/{data_fim[1]}/{data_fim[2]}&txUF={UF}&txSessao=&listaTipoSessao=&listaTipoInterv=&inFalaPres=&listaTipoFala=&listaFaseSessao=&txAparteante=&listaEtapa=&CampoOrdenacao=dtSessao&TipoOrdenacao=ASC&PageSize={tamanho}&txTexto=&txSumario='
        soup.append(UrlToSoup(url))
        for links in soup[i].find_all('a', attrs={'title':"Íntegra do Discurso"}):
            temp.append((links['href']))
        link_infos.append(temp)
        temp = []


In [ ]:
link_infos

In [174]:
len(link_infos[0])

100

In [176]:
linkDiscurso = []
temp = []
for i in range(qpt):
    for item in link_infos[i]:
        text = item.replace('\t','').replace('\n','').replace('\r','').replace(' ','')
        temp.append(("https://www.camara.leg.br/internet/SitaqWeb/"+text))
    linkDiscurso.append(temp)
    temp = []

In [179]:
len(linkDiscurso[0])

100

In [183]:
len(linkDiscurso[0])

100

In [184]:
linkDiscurso

[['https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuInsercao=3&dtHorarioQuarto=12:37&sgFaseSessao=OD&Data=28/04/2022&txApelido=AFONSOFLORENCE,PT-BA&txFaseSessao=OrdemdoDia&txTipoSessao=Extraordinária-CN&dtHoraQuarto=12:37&txEtapa=',
  'https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuInsercao=23&dtHorarioQuarto=12:37&sgFaseSessao=OD&Data=28/04/2022&txApelido=AFONSOFLORENCE,PT-BA&txFaseSessao=OrdemdoDia&txTipoSessao=Extraordinária-CN&dtHoraQuarto=12:37&txEtapa=',
  'https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuInsercao=27&dtHorarioQuarto=12:37&sgFaseSessao=OD&Data=28/04/2022&txApelido=AFONSOFLORENCE,PT-BA&txFaseSessao=OrdemdoDia&txTipoSessao=Extraordinária-CN&dtHoraQuarto=12:37&txEtapa=',
  'https://www.camara.leg.br/internet/SitaqWeb/TextoHTML.asp?etapa=5&nuSessao=013.4.56.N&nuQuarto=0&nuOrador=0&nuIns